In [ ]:
## Explore and sample stock headlines/blurbs and review the model's ability to evaluate the sentiment.
## The purpose of this notebook is to obtain the results of webscraping news headlines and blurbs related to a stock symbol.
## and to evaluate how the model regards the sentiment each text.

In [3]:
import requests
import urllib.request
import time
import spacy
from bs4 import BeautifulSoup
import pandas as pd
from joblib import dump, load

In [4]:
clf = load('sentiment.joblib')

In [53]:
stock_name = "BAC"

In [54]:
numResults = 100
url = f'https://www.google.com/search?q=stock+symbol+{stock_name}&tbm=nws&hl=en&num={str(numResults)}'
# https://www.google.com/search?q=stock+symbol+AMZN&tbm=nws&hl=en&num=50

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Get the links, headline and description text
results = soup.find_all("div", attrs={"class": "ZINbbc"})
headlines = []
descriptions = []
links = []
for result in results:
    try:
        headline = result.find("div", attrs={"class": "vvjwJb"}).get_text()
        if headline != "":
            headlines.append(headline)
            links.append("google.com"+result.a["href"])
        description = result.find(
            "div", attrs={"class": "s3v9rd"}).get_text()
        if description != "":
            # remove the "1 day ago" etc.
            description = description[description.find("·")+1:]
            descriptions.append(description)
    except:
        continue

In [55]:
text_list = []
for i in range(0,len(headlines)-1):
    text_list.append(headlines[i]+" "+descriptions[i])

In [56]:
text_list

['February 2022 Options Now Available For Bank of America (BAC)  ... in Bank of America Corp (Symbol: BAC) saw new options begin trading ... If an investor was to purchase shares of BAC stock at the current...',
 'BAC July 30th Options Begin Trading  Investors in Bank of America Corp (Symbol: BAC) saw new options begin trading today, for the July 30th expiration. At Stock Options Channel,...',
 'Why Astra Space Stock Just Dropped 10%  10 stocks we like better than Astra Space, Inc. ... And Bank of America Corp (Symbol: BAC) options are showing a volume of 232,034 contracts...',
 'Buy WISH Stock Before It Joins GME and AMC as a Retail Favorite  WISH Stock has been jumping for a while now due to meme mania. ... And Bank of America Corp (Symbol: BAC) options are showing a volume...',
 "Wall Street and Your Fiends Both Like SoFi, So Buy SOFI Stock  Everyone we've told about SOFI stock loves that too. ... heat to banks like Wells Fargo (NYSE:WFC), Bank of America (NYSE:BAC) and others.",
 '

In [57]:
df = pd.DataFrame({"News": text_list,
                   "sentiment_no": [clf.predict([text])[0] for text in text_list]})
df

,News,sentiment_no
0,February 2022 Options Now Available For Bank o...,0
1,BAC July 30th Options Begin Trading Investors...,1
2,Why Astra Space Stock Just Dropped 10% 10 sto...,0
3,Buy WISH Stock Before It Joins GME and AMC as ...,0
4,"Wall Street and Your Fiends Both Like SoFi, So...",1
...,...,...
94,Quicken Loans parent company's stock skyrocket...,1
95,Thinking about trading options or stock in Nio...,1
96,Diamond Hill Capital Trades Citigroup for Bank...,1
97,"The Zacks Analyst Blog Highlights: JPMorgan, B...",1


In [58]:
df.sentiment_no.value_counts()

0    53
1    46
Name: sentiment_no, dtype: int64

In [59]:
df.sample(13).to_csv("stock_text_sample.csv")